# Use vitessce-image-viewer in Jupyter Notebooks
(Revision: 0.2.2)

## Hello world for ImJoy

In [ ]:
from imjoy import api

class ImJoyPlugin():
    async def setup(self):
        pass

    async def run(self, ctx):
        await api.alert("hello world")

api.export(ImJoyPlugin())

## Run the vitessce-image-viewer

In [ ]:
from imjoy import api

class ImJoyPlugin():
    async def setup(self):
        pass

    async def run(self, ctx):
        viewer = await api.createWindow(type="vitessce-image-viewer", src="https://oeway.github.io/vitessce-image-viewer/")

api.export(ImJoyPlugin())

## You can also run it as a dialog

In [ ]:
api.showDialog(type="vitessce-image-viewer", src="https://oeway.github.io/vitessce-image-viewer/")

## Download example data
(From Nicholas Sofroniew's Napari training course: https://github.com/sofroniewn/napari-training-course)

In [ ]:
import urllib.request
urllib.request.urlretrieve(
    'https://github.com/sofroniewn/napari-training-course/raw/master/lessons/data/cell_membranes.tif', 
    'cell_membranes.tif')

In [ ]:
import zarr
from tifffile import imread
image = imread('cell_membranes.tif')
image = (image*65536).astype('uint16')
z_array = zarr.array(image, chunks=(50, 50))
z_array.shape

## Register a  imjoy-rpc codec to encode zarr array/group

This is to allow us sending Zarr array or group directly to the viewer in JS.
In the viewer, Zarr.js will use the encoded object as a custom store.

This allows us send zarr array in a lazy way, the data can be requested on demand.

In [ ]:
from imjoy import api

def encode_zarr_store(zobj):
    def getItem(key):
        return zobj.store[key]

    def setItem(key, value):
        zobj.store[key] = value

    def containsItem(key):
        return key in zobj.store

    return {
        "_rintf": True,
        "_rtype": 'zarr-array' if isinstance(zobj, zarr.Array) else 'zarr-group',
        "getItem": getItem,
        "setItem": setItem,
        "containsItem": containsItem,
    }

api.registerCodec({'name': 'zarr-array', 'type': zarr.Array, "encoder": encode_zarr_store})
api.registerCodec({'name': 'zarr-group', 'type': zarr.hierarchy.Group, "encoder": encode_zarr_store})

## Prepare the object for the viewer

In [ ]:
staticInfo = {
  # here we pass directly the zarr array, because we have registered a codec
  "url": z_array,
  "isPublic": False,
  "initialViewState": {
    "zoom": -1,
    "target": [100, 100]
  },
  "dimensions": [
    {
      "field": 'mz',
      "type": 'ordinal',
      "values": [str(i) for i in range(60)]
    },
    { "field": 'y', "type": 'quantitative', "values": None },
    { "field": 'x', "type": 'quantitative', "values": None }
  ],
  'selections': [{'mz': 21}],
    'colormap': 'viridis',
  "description": 'Membrane (zarr)'
}

In [ ]:
from imjoy import api
import asyncio

class ImJoyPlugin():
    async def setup(self):
        pass

    async def run(self, ctx):
        viewer = await api.createWindow(type="vitessce-image-viewer", src="https://oeway.github.io/vitessce-image-viewer/")
        #await asyncio.sleep(10)
        await viewer.loadSource('static', staticInfo)

api.export(ImJoyPlugin())

## Example Zarr dataset: Kidney IMS (zarr)

In [ ]:
import zarr
from fsspec.implementations.http import HTTPFileSystem
fs = HTTPFileSystem()
http_map = fs.get_mapper('https://vitessce-data.storage.googleapis.com/0.0.25/master_release/spraggins/spraggins.ims.zarr')
z_array = zarr.open_array(http_map, mode='r')

In [ ]:
staticInfo = {
  # here we pass directly the zarr array, because we have registered a codec
  "url": z_array,
  "isPublic": False,
  "initialViewState": {
    "zoom": -1,
    "target": [1000, 500]
  },
  "dimensions": [
    {
      "field": 'mz',
      "type": 'ordinal',
      "values": [
        '675.5366',
        '703.5722',
        '721.4766',
        '725.5562',
        '729.5892',
        '731.606',
        '734.5692',
        '737.4524',
        '739.4651',
        '741.5302',
        '745.4766',
        '747.4938',
        '749.5093',
        '753.5892',
        '756.5534',
        '758.5706',
        '772.5225',
        '772.5506',
        '776.5928',
        '780.5528',
        '782.5697',
        '784.5841',
        '786.6012',
        '787.6707',
        '790.5157',
        '796.5259',
        '798.54',
        '804.5528',
        '806.5683',
        '808.5838',
        '809.6518',
        '810.6',
        '811.6699',
        '813.6847',
        '815.699',
        '820.5262',
        '822.5394',
        '824.5559',
        '825.6241',
        '828.5495',
        '830.5666',
        '832.5816',
        '833.649',
        '835.6666',
        '837.6798',
        '848.5577',
        '851.6374'
      ]
    },
    { "field": 'y', "type": 'quantitative', "values": None },
    { "field": 'x', "type": 'quantitative', "values": None }
  ],
  'selections': [{ "mz": '703.5722' }, { "mz": '721.4766' }],
  "description": 'Kidney IMS (zarr)'
}

In [ ]:
from imjoy import api
import asyncio

class ImJoyPlugin():
    async def setup(self):
        pass

    async def run(self, ctx):
        viewer = await api.createWindow(type="vitessce-image-viewer", src="https://oeway.github.io/vitessce-image-viewer/")
        #await asyncio.sleep(10)
        await viewer.loadSource('static', staticInfo)

api.export(ImJoyPlugin())

## Example Zarr dataset: Kidney mxIF (zarr)

In [ ]:
import zarr
from fsspec.implementations.http import HTTPFileSystem
fs = HTTPFileSystem()
http_map = fs.get_mapper('https://vitessce-data.storage.googleapis.com/0.0.25/master_release/spraggins/spraggins.mxif.zarr')

z_group = zarr.open_group(http_map, mode="r")
# z = zarr.open_array(http_map, path="0/", mode='r')

In [ ]:

channelNames = [
  'DAPI - Hoechst (nuclei)',
  'FITC - Laminin (basement membrane)',
  'Cy3 - Synaptopodin (glomerular)',
  'Cy5 - THP (thick limb)'
]


channelNames = [
  'DAPI - Hoechst (nuclei)',
  'FITC - Laminin (basement membrane)',
  'Cy3 - Synaptopodin (glomerular)',
  'Cy5 - THP (thick limb)'
]


zarrInfo = {
  # here we pass directly the zarr group, because we have registered a codec
  "url": z_group,
  "description": 'Kidney mxIF (zarr)',
  "dimensions": [
    { "field": 'channel', "type": 'nominal', "values": channelNames },
    { "field": 'y', "type": 'quantitative', "values": None },
    { "field": 'x', "type": 'quantitative', "values": None }
  ],
  "isPublic": True,
  "isPyramid": True,
  "selections": [{ "channel": name } for name in channelNames]
}
    


In [ ]:
from imjoy import api
import asyncio

class ImJoyPlugin():
    async def setup(self):
        pass

    async def run(self, ctx):
        viewer = await api.createWindow(type="vitessce-image-viewer", src="https://oeway.github.io/vitessce-image-viewer/")
        #await asyncio.sleep(10)
        await viewer.loadSource('static', zarrInfo)

api.export(ImJoyPlugin())

## Example Zarr dataset: IDR 9822151.zarr - SARS-CoV-2 in human instestinal cells (OME-ZARR)

In [ ]:
import zarr
from fsspec.implementations.http import HTTPFileSystem
fs = HTTPFileSystem()
http_map = fs.get_mapper('https://vitessce-demo-data.storage.googleapis.com/test-data/9822151.zarr')

z_group = zarr.open_group(http_map, mode="r")

In [ ]:
omeZarr = {
   # here we pass directly the zarr group, because we have registered a codec
  "url": z_group,
  "dimensions": [
    # TODO: Having the actual dimensions breaks the UI components currently
    # ome_zarr images are all (t, c, z, y, x)
    { "field": 'c', "type": 'nominal', "values": [0] },
    { "field": 'y', "type": 'quantitative', "values": None },
    { "field": 'x', "type": 'quantitative', "values": None }
  ],
  "selections": [{ "c": 0 }],
  "isPublic": True,
  "isPyramid": True,
  "description":
    'IDR 9822151.zarr - SARS-CoV-2 in human instestinal cells (OME-ZARR)',
  "initialViewState": {
    "zoom": -5,
    "target": [30000, 20000]
  }
}

In [ ]:
from imjoy import api
import asyncio

class ImJoyPlugin():
    async def setup(self):
        pass

    async def run(self, ctx):
        viewer = await api.createWindow(type="vitessce-image-viewer", src="https://oeway.github.io/vitessce-image-viewer/")
        #await asyncio.sleep(10)
        await viewer.loadSource('ome-zarr', omeZarr)

api.export(ImJoyPlugin())